In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path

2023-10-17 13:44:10.630426: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("train*.tfrecords"))

In [3]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['gender']
    return ecg_data, label

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

2023-10-17 13:44:17.123973: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46582 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6


In [8]:
def conv_block(x, n, k, mp):
    x = tf.keras.layers.Conv1D(n, k, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    if mp:
        x = tf.keras.layers.MaxPooling1D(mp)(x)

    return x

input_layer = tf.keras.layers.Input(shape=(5000,12))
x = input_layer

x = conv_block(x, 16, 7, 2)
x = conv_block(x, 16, 5, 4)
x = conv_block(x, 32, 5, 2)
x = conv_block(x, 32, 5, 4)
x = conv_block(x, 64, 5, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)

x = tf.keras.layers.Conv1D(128, 12, padding = 'same', data_format = 'channels_first')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling1D()(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(input_layer, x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

In [9]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

In [10]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(make_checkpoint_dir("data/models", "cnn-gender"))
]

model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10


2023-10-17 13:44:58.472192: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f714e8b70b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-17 13:44:58.472248: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2023-10-17 13:44:58.655256: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-17 13:44:59.682749: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   7851/Unknown - 173s 21ms/step - loss: 0.5080 - accuracy: 0.7520 - auc_1: 0.8307

2023-10-17 13:47:47.407237: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 3989729919823237703
2023-10-17 13:47:47.407319: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 16579930753945695078
2023-10-17 13:49:12.753650: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 3989729919823237703
2023-10-17 13:49:12.753714: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 16579930753945695078


INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-17_13-44-53/assets


...

7851/7851 [==============================] - 248s 32ms/step - loss: 0.3940 - accuracy: 0.8253 - auc_1: 0.9038 - val_loss: 0.4281 - val_accuracy: 0.8056 - val_auc_1: 0.8886 - lr: 0.0010
Epoch 9/10
7850/7851 [============================>.] - ETA: 0s - loss: 0.3873 - accuracy: 0.8286 - auc_1: 0.9072INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-17_13-44-53/assets


INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-17_13-44-53/assets


7851/7851 [==============================] - 248s 32ms/step - loss: 0.3873 - accuracy: 0.8286 - auc_1: 0.9072 - val_loss: 0.4294 - val_accuracy: 0.8055 - val_auc_1: 0.8879 - lr: 0.0010
Epoch 10/10
7851/7851 [==============================] - ETA: 0s - loss: 0.3828 - accuracy: 0.8314 - auc_1: 0.9095INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-17_13-44-53/assets


INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-17_13-44-53/assets


7851/7851 [==============================] - 267s 34ms/step - loss: 0.3828 - accuracy: 0.8314 - auc_1: 0.9095 - val_loss: 0.4478 - val_accuracy: 0.8044 - val_auc_1: 0.8870 - lr: 0.0010


In [11]:
model.save("data/models/cnn-gender.keras")